In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
import os
import cv2
from sklearn.model_selection import train_test_split
from tqdm import tqdm  # import tqdm for progress bar 
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [2]:
target_labels = ['Cherry_(including_sour)___healthy', 'Cherry_(including_sour)___Powdery_mildew', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___healthy', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)']

In [3]:
main_path = 'data_penyakit/dataset/train'

In [4]:
# Load the images and labels
filename = []
X = []
y = []
for label in target_labels:
    label_dir = os.path.join(main_path, label)
    for img_file in os.listdir(label_dir):
        img_path = os.path.join(label_dir, img_file)
        img = load_img(img_path, target_size=(128, 128))
        img_array = img_to_array(img)
        filename.append(img_file)
        X.append(img_array)
        y.append(target_labels.index(label))

In [5]:
from tensorflow.keras.utils import to_categorical

In [6]:
X = np.array(X)
y = to_categorical(y, num_classes=len(target_labels))

In [7]:
X.shape

(10731, 128, 128, 3)

In [8]:
y.shape

(10731, 6)

In [9]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test, filename_train, filename_test = train_test_split(X, y, filename, test_size=0.2, random_state=42)

In [10]:
X_train.shape

(8584, 128, 128, 3)

In [11]:
X_test.shape

(2147, 128, 128, 3)

In [12]:
y_train.shape

(8584, 6)

In [13]:
y_test.shape

(2147, 6)

In [14]:
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.models import Sequential

In [15]:
len(target_labels)

6

In [16]:
cnn_model = Sequential()
cnn_model.add(Conv2D(32, kernel_size= (3,3), activation = 'relu',input_shape=(128,128,3)))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(64,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(64,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(96,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(32,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())

cnn_model.add(Dropout(0.2))
cnn_model.add(Flatten())
cnn_model.add(Dense(128, activation = 'relu'))
cnn_model.add(Dropout(0.3))
cnn_model.add(Dense(len(target_labels), activation='softmax'))
cnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
# Train the CNN model
cnn_history = cnn_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10)

Epoch 1/10


2023-05-31 13:07:28.633166: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


269/269 [==============================] - 112s 411ms/step - loss: 0.4544 - accuracy: 0.8324 - val_loss: 0.8099 - val_accuracy: 0.7182
Epoch 2/10
269/269 [==============================] - 116s 432ms/step - loss: 0.1245 - accuracy: 0.9560 - val_loss: 0.2652 - val_accuracy: 0.9003
Epoch 3/10
269/269 [==============================] - 120s 446ms/step - loss: 0.0921 - accuracy: 0.9678 - val_loss: 0.1878 - val_accuracy: 0.9301
Epoch 4/10
269/269 [==============================] - 114s 423ms/step - loss: 0.0635 - accuracy: 0.9790 - val_loss: 0.1200 - val_accuracy: 0.9581
Epoch 5/10
269/269 [==============================] - 118s 438ms/step - loss: 0.0402 - accuracy: 0.9877 - val_loss: 1.1274 - val_accuracy: 0.7387
Epoch 6/10
269/269 [==============================] - 83s 308ms/step - loss: 0.0377 - accuracy: 0.9870 - val_loss: 0.2610 - val_accuracy: 0.9152
Epoch 7/10
269/269 [==============================] - 70s 259ms/step - loss: 0.0512 - accuracy: 0.9832 - val_loss: 0.0455 - val_accuracy

In [18]:
# Evaluate the performance of the trained CNN model on the test set
cnn_scores = cnn_model.evaluate(X_test, y_test, verbose=0)
print("CNN Model Accuracy: %.2f%%" % (cnn_scores[1] * 100))

CNN Model Accuracy: 99.39%


In [19]:
cnn_scores

[0.018038498237729073, 0.9939450621604919]

In [20]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Use the model to predict the test dataset
y_pred_prob = cnn_model.predict(X_test)

# Convert probabilities to classes
y_pred = np.argmax(y_pred_prob, axis=1)

# If y_test is one-hot encoded, convert it to classes as well
y_test_classes = np.argmax(y_test, axis=1)

# Calculate accuracy
accuracy = accuracy_score(y_test_classes, y_pred)

# Calculate precision
precision = precision_score(y_test_classes, y_pred, average='weighted') 

# Calculate recall
recall = recall_score(y_test_classes, y_pred, average='weighted') 

# Calculate F1 score
f1 = f1_score(y_test_classes, y_pred, average='weighted')

# Print the metrics
print("Accuracy: %.2f%%" % (accuracy * 100))
print("Precision: %.2f%%" % (precision * 100))
print("Recall: %.2f%%" % (recall * 100))
print("F1 score: %.2f%%" % (f1 * 100))

68/68 [==============================] - 9s 127ms/step
Accuracy: 99.39%
Precision: 99.40%
Recall: 99.39%
F1 score: 99.39%


In [21]:
from tabulate import tabulate

# Create a DataFrame with true labels, predicted labels, and target labels
df = pd.DataFrame({'Filename': filename_test,
                   'True Labels': [target_labels[label] for label in y_test_classes],
                   'Predicted Labels': [target_labels[label] for label in y_pred]})

# Convert DataFrame to tabular format
table = tabulate(df, headers='keys', tablefmt='psql')

# Print the table
print(table)

# Save the DataFrame to a CSV file
df.to_csv('labels_predictions-cnn-cerry+anggur.csv', index=False)

+------+--------------------------------------------------------------------------+--------------------------------------------+--------------------------------------------+
|      | Filename                                                                 | True Labels                                | Predicted Labels                           |
|------+--------------------------------------------------------------------------+--------------------------------------------+--------------------------------------------|
|    0 | 77df2a22-77fe-4617-83c4-f37e1776e225___JR_HL 9665.JPG                    | Cherry_(including_sour)___healthy          | Cherry_(including_sour)___healthy          |
|    1 | 33ebcc8a-b2a9-4ae0-b95a-670e01b81c01___JR_HL 9694_180deg.JPG             | Cherry_(including_sour)___healthy          | Cherry_(including_sour)___healthy          |
|    2 | 9157434b-8dce-4726-9187-1911302312b9___JR_HL 4053.JPG                    | Cherry_(including_sour)___healthy          | C

In [22]:
# Filter rows where true label and predicted label do not match
false_predictions = df[df['True Labels'] != df['Predicted Labels']]

# Convert DataFrame to tabular format
table = tabulate(false_predictions, headers='keys', tablefmt='psql')

# Print the false predictions table
print("False Predictions:")
print(table)

# Save the false predictions to a CSV file
false_predictions.to_csv('false_predictions-cnn-cerry+anggur.csv', index=False)

False Predictions:
+------+---------------------------------------------------------------------+--------------------------------------------+--------------------------------------------+
|      | Filename                                                            | True Labels                                | Predicted Labels                           |
|------+---------------------------------------------------------------------+--------------------------------------------+--------------------------------------------|
|   83 | 30e108e7-3a95-401f-a815-11397d206839___FAM_B.Rot 0548.JPG           | Grape___Black_rot                          | Grape___Esca_(Black_Measles)               |
|  403 | 594fbe5a-a79c-44e6-a4b8-86064bcc5115___FAM_L.Blight 1465.JPG        | Grape___Leaf_blight_(Isariopsis_Leaf_Spot) | Grape___healthy                            |
|  547 | 669168c0-ff03-4708-a10a-153d5cd1e10f___FAM_B.Rot 3481.JPG           | Grape___Black_rot                          | Grape___Esca

In [23]:
# Extract the features using the trained CNN model
cnn_features = cnn_model.predict(X_train)

269/269 [==============================] - 42s 158ms/step


# rf

In [24]:
import time

In [25]:
# import the Random Forest classifier
from RF_Manual import RandomForest

In [26]:
# Train a Random Forest classifier on the extracted features
start_time = time.time()
rf_classifier = RandomForest(n_trees=100, max_depth=42)
rf_classifier.fit(cnn_features, np.argmax(y_train, axis=1))

# count time for training
end_time = time.time()
training_time = end_time - start_time
print(f"Training time: {training_time} seconds")

Training time: 361.47267508506775 seconds


In [27]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Evaluate the performance of the Random Forest classifier on the test set
rf_features = cnn_model.predict(X_test)
rf_features = rf_features.reshape(rf_features.shape[0], -1)
rf_predictions = rf_classifier.predict(rf_features)
rf_accuracy = np.mean(rf_predictions == np.argmax(y_test, axis=1))
print("Random Forest Classifier Accuracy: %.2f%%" % (rf_accuracy * 100))

# Calculate and print the Precision
rf_precision = precision_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier Precision: %.2f" % rf_precision)

# Calculate and print the Recall
rf_recall = recall_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier Recall: %.2f" % rf_recall)

# Calculate and print the F1 score
rf_f1_score = f1_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier F1 Score: %.2f" % rf_f1_score)

68/68 [==============================] - 12s 179ms/step
Random Forest Classifier Accuracy: 99.77%
Random Forest Classifier Precision: 1.00
Random Forest Classifier Recall: 1.00
Random Forest Classifier F1 Score: 1.00


In [28]:
# Create a DataFrame with true labels and predicted labels
df_rf = pd.DataFrame({'Filename': filename_test,
                      'True Labels': [target_labels[label] for label in y_test_classes],
                      'Predicted Labels': [target_labels[label] for label in rf_predictions]})

# Convert DataFrame to tabular format
table_rf = tabulate(df_rf, headers='keys', tablefmt='psql')

# Print the table
print(table_rf)

# Save the DataFrame to a CSV file
df_rf.to_csv('labels_predictions-rf-cerry+anggur.csv', index=False)

+------+--------------------------------------------------------------------------+--------------------------------------------+--------------------------------------------+
|      | Filename                                                                 | True Labels                                | Predicted Labels                           |
|------+--------------------------------------------------------------------------+--------------------------------------------+--------------------------------------------|
|    0 | 77df2a22-77fe-4617-83c4-f37e1776e225___JR_HL 9665.JPG                    | Cherry_(including_sour)___healthy          | Cherry_(including_sour)___healthy          |
|    1 | 33ebcc8a-b2a9-4ae0-b95a-670e01b81c01___JR_HL 9694_180deg.JPG             | Cherry_(including_sour)___healthy          | Cherry_(including_sour)___healthy          |
|    2 | 9157434b-8dce-4726-9187-1911302312b9___JR_HL 4053.JPG                    | Cherry_(including_sour)___healthy          | C

In [29]:
# Filter rows where true label and predicted label do not match
false_predictions_rf = df_rf[df_rf['True Labels'] != df_rf['Predicted Labels']]

# Convert DataFrame to tabular format
table_rf = tabulate(false_predictions_rf, headers='keys', tablefmt='psql')

# Print the false predictions table
print("False Predictions:")
print(table_rf)

# Save the false predictions to a CSV file
false_predictions_rf.to_csv('false_predictions-rf-cerry+anggur.csv', index=False)

False Predictions:
+------+------------------------------------------------------------+------------------------------+------------------------------+
|      | Filename                                                   | True Labels                  | Predicted Labels             |
|------+------------------------------------------------------------+------------------------------+------------------------------|
|   83 | 30e108e7-3a95-401f-a815-11397d206839___FAM_B.Rot 0548.JPG  | Grape___Black_rot            | Grape___Esca_(Black_Measles) |
|  842 | 0e1c89b9-d289-471a-988f-850f4874621e___FAM_B.Msls 3950.JPG | Grape___Esca_(Black_Measles) | Grape___Black_rot            |
| 1471 | cd7bc6d0-0a45-4d7f-8474-1cb3bfc8adba___FAM_B.Rot 3268.JPG  | Grape___Black_rot            | Grape___Esca_(Black_Measles) |
| 1703 | 7b2ca8d1-8192-4009-920d-06b5ec7378c0___FAM_B.Rot 3266.JPG  | Grape___Black_rot            | Grape___Esca_(Black_Measles) |
| 1895 | 8a92c94e-8128-4c39-9ead-eebf6f3fe74d___FAM_B.Rot